# Implementing ANN

In [28]:
import os
import numpy as np
import tensorflow as tf
import keras

from PIL import Image

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import TensorBoard

from time import strftime # gives hours and minutes of current time.

from sklearn.metrics import precision_score, recall_score

# import matplotlib.pyplot as plt

In [2]:
print(tf.__version__, keras.__version__)

2.13.0 2.13.1


### Loading the data

In [3]:
df_images = np.load('../3_PreprocessingMaleDS/df_images.npy')

print(type(df_images))
df_images.shape

<class 'numpy.ndarray'>


(10842, 80, 60, 3)

In [4]:
df_usageEncoded = np.load('../3_PreprocessingMaleDS/df_usageEncoded.npy')

print(type(df_usageEncoded))
df_usageEncoded.shape

<class 'numpy.ndarray'>


(10842,)

### Scaling and flattening the data

In [5]:
#Scaling the data between 0 and 1 (Normalization)
x_scaled = df_images / 255.0

In [6]:
#Let us see the data again after scaling.
x_scaled
#note that this is a 4D tensor.

array([[[[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        ...,

        [[1.        , 1.        , 0.97254902],
         [1.        , 1.        , 0.98039216]

In [7]:
#Flattening the train tensor; placing all pixel for one image in one dimension
TOTAL_INPUTS = 80*60*3
x_scaled_flat = x_scaled.reshape(x_scaled.shape[0], TOTAL_INPUTS)

In [8]:
#Let us see how this flat array looks like
x_scaled_flat
#Now it is a 2D tensor.

array([[1.        , 1.        , 1.        , ..., 0.99607843, 1.        ,
        0.99215686],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.76862745, 0.75686275, 0.72941176, ..., 0.74901961, 0.7372549 ,
        0.70980392],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.76470588, 0.75294118, 0.7254902 , ..., 0.74901961, 0.7372549 ,
        0.71764706]])

In [9]:
x_scaled_flat.shape
#Note that 14400 = 80 x 60 x 3

(10842, 14400)

### Creating train, test and validation Dataset

In [10]:
TEST_SIZE = 1000

##Creating test set
x_test = x_scaled_flat[:TEST_SIZE]
y_test = df_usageEncoded[:TEST_SIZE]
x_test.shape

(1000, 14400)

In [11]:
VAL_SIZE = 500

##Creating test set
x_val = x_scaled_flat[:VAL_SIZE]
y_val = df_usageEncoded[:VAL_SIZE]
x_val.shape

(500, 14400)

In [12]:
##Creating the remaining train set
x_train = x_scaled_flat[TEST_SIZE + VAL_SIZE:]
y_train = df_usageEncoded[TEST_SIZE + VAL_SIZE:]
x_train.shape

(9342, 14400)

So now we have two scaled and flattened datasets:
- The train set haing 15366 samples
- The test set having 5000 samples
- The validation set having 1000 samples

## Define the Neural Network using Keras

### Model 1

In [13]:
model_1 = Sequential([
    Dense(units=128, input_dim=TOTAL_INPUTS, activation='relu', name='m1_hidden1'),
    Dense(units=64, activation='relu', name='m1_hidden2'),
    Dense(16, activation='relu', name='m1_hidden3'),
    Dense(2, activation='softmax', name='m1_output')
])
#if we donot give names to the layers, then the names keep on changing on every run

model_1.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])

In [14]:
type(model_1)
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 m1_hidden1 (Dense)          (None, 128)               1843328   
                                                                 
 m1_hidden2 (Dense)          (None, 64)                8256      
                                                                 
 m1_hidden3 (Dense)          (None, 16)                1040      
                                                                 
 m1_output (Dense)           (None, 2)                 34        
                                                                 
Total params: 1852658 (7.07 MB)
Trainable params: 1852658 (7.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


- Total neurons in layer m1_hidden1 = (TOTAL_INPUTS+1)*128 = ((80*60*3)+1)*128 = 1843328
- Total neurons in layer m1_hidden2 = (128+1)*64  
- Total neurons in layer m1_hidden3 = (64+1)*16
- Total neurons in layer m1_output = (16+1)*10

## Tensorboard (visualising learning)

In [15]:
#Setting main folder and subfolders for tendboard
LOG_DIR = 'tensorboard_cifar_logs/'

def get_tensorboard(model_name):
    sub_folder_name = f'{model_name}_at_{strftime("%H_%M")}'
    dir_paths = os.path.join(LOG_DIR, sub_folder_name)
    os.makedirs(dir_paths)
    return TensorBoard(log_dir=dir_paths)

### Loading tensor board in notebook

In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir=tensorboard_cifar_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9896), started 0:00:04 ago. (Use '!kill 9896' to kill it.)

## Fitting the Model

In [18]:
samples_per_batch = 1000
nr_epochs = 150

In [19]:
%%time
model_1.fit(x_train, y_train, batch_size=samples_per_batch, epochs=nr_epochs,
            callbacks=[get_tensorboard('Model_1')], verbose=0, validation_data=(x_val, y_val))

CPU times: total: 7min 52s
Wall time: 3min 26s


In [20]:
# Saving the model
model_1.save('ANN_2Hidden.h5')

C:\Users\Electro Store\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Making Predictions on Individual Images

- In the following code model_1 is used for prediction.
- You may use model_2 or model_3 too by making necessary alterations in the model name.

In [21]:
image_nr=25
x_val[image_nr].shape

(14400,)

In [22]:
##Adding a dimension as per requirement of predict method
test = np.expand_dims(x_val[image_nr], axis=0)
test.shape

(1, 14400)

In [23]:
model_1.predict(test)

1/1 [==============================] - 0s 122ms/step


array([[0.8082922 , 0.19170776]], dtype=float32)

In [24]:
#Picking the highest probability class
predicted_value=np.argmax(model_1.predict(test), axis=1)
actual_value=y_val[image_nr]

print(f'Actual value: {actual_value} vs. predicted: {predicted_value[0]}')

1/1 [==============================] - 0s 36ms/step
Actual value: 0 vs. predicted: 0


## Evaluation

In [25]:
#Recalling the metrics that we set during compilation of the model.
model_1.metrics_names

['loss', 'accuracy']

In [26]:
# Let us print the loss funcstion value and overall accuracy of our model on test data.
test_loss, test_accuracy = model_1.evaluate(x_test, y_test)
print(f'Test loss is {test_loss:0.3} and test accuracy is {test_accuracy:0.1%}')

32/32 [==============================] - 0s 5ms/step - loss: 0.1612 - accuracy: 0.9440
Test loss is 0.161 and test accuracy is 94.4%


In [41]:
y_pred =np.argmax(model_1.predict(x_test), axis=1)

32/32 [==============================] - 0s 5ms/step


In [42]:
precision = precision_score(y_test, y_pred)
precision

0.8631578947368421

In [44]:
recall = recall_score(y_test, y_pred)
recall

0.656

### Predicting real-world data

In [3]:
im = Image.open('./formalShirt2.jpg')
im = im.resize((80, 60))

im_arr = np.array(im)
scaled_imArr = im_arr/255
flat_imArr = scaled_imArr.reshape(-1)

im = np.expand_dims(flat_imArr, axis=0)

model = tf.keras.models.load_model('./ANN_2Hidden.h5')

pred1 = model.predict(im)

print(type(pred1))
print(pred1)
print(np.max(pred1))
np.argmax(pred1)

1/1 [==============================] - 0s 109ms/step
<class 'numpy.ndarray'>
[[0.8674154  0.13258463]]
0.8674154


0

In [2]:
im2 = Image.open('./casualShirt.jpg')
im2 = im2.resize((80, 60))

im2_arr = np.array(im2)
scaled_im2Arr = im2_arr/255
flat_im2Arr = scaled_im2Arr.reshape(-1)

im2 = np.expand_dims(flat_im2Arr, axis=0)

model = tf.keras.models.load_model('./ANN_2Hidden.h5')

pred2 = model.predict(im2)

print(type(pred2))
print(pred2)
print(np.max(pred2))
np.argmax(pred2)

1/1 [==============================] - 0s 167ms/step
<class 'numpy.ndarray'>
[[0.99579805 0.00420195]]
0.99579805


0